# CLIP + ImageNet 图像分类示例

本笔记本演示如何使用自定义的ImageNet数据集类进行CLIP图像分类。

## 数据集类特点

✅ **继承自 `torch.utils.data.Dataset`**  
✅ **自动解析WNID和类别名称**  
✅ **支持CLIP预处理**  
✅ **灵活的数据增强**  
✅ **类别子集选择**  
✅ **自动处理train/val分割**

## 1. 导入依赖

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import transforms
import numpy as np

# 导入自定义数据集类
from imagenet_dataset import ImageNetDataset, create_imagenet_dataloaders, get_transforms

# 设置matplotlib中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA设备: {torch.cuda.get_device_name(0)}")

## 2. 数据加载

### 2.1 使用便捷函数创建DataLoader

In [ ]:
# 配置参数
DATA_ROOT = r"G:\Thomas\3_1_project\data\ImageNet-data"
BATCH_SIZE = 16
NUM_CLASSES = 20  # 使用前20个类别
IMAGE_SIZE = 224
SEED = 42

print("创建数据加载器...")
print("="*70)

train_loader, val_loader = create_imagenet_dataloaders(
    root=DATA_ROOT,
    batch_size=BATCH_SIZE,
    num_workers=0,  # Windows系统建议设为0
    num_classes=NUM_CLASSES,
    image_size=IMAGE_SIZE,
    use_clip_norm=True,  # 使用CLIP的归一化参数
    seed=SEED,
    val_split=0.1,  # 10%作为验证集
)

print(f"\n✓ 训练集批次数: {len(train_loader)}")
print(f"✓ 验证集批次数: {len(val_loader)}")

# 获取基础dataset（处理可能的Subset包装）
base_dataset = train_loader.dataset
if hasattr(base_dataset, 'dataset'):
    base_dataset = base_dataset.dataset

print(f"\n数据集统计:")
print(f"  类别数: {base_dataset.num_classes}")
print(f"  图像尺寸: {IMAGE_SIZE}x{IMAGE_SIZE}")
print(f"  归一化方式: CLIP")

### 2.2 查看类别信息

In [ ]:
print("ImageNet类别列表:")
print("="*70)

for i in range(base_dataset.num_classes):
    wnid = base_dataset.get_wnid(i)
    name = base_dataset.get_class_name(i)
    print(f"  [{i:2d}] {wnid}: {name}")

## 3. 数据可视化

### 3.1 反归一化函数

In [ ]:
def denormalize_clip(tensor):
    """
    反归一化CLIP图像
    
    Args:
        tensor: 归一化后的图像张量 [C, H, W] 或 [B, C, H, W]
    
    Returns:
        反归一化后的图像
    """
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073]).view(3, 1, 1)
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711]).view(3, 1, 1)
    
    if tensor.dim() == 4:  # batch
        mean = mean.unsqueeze(0)
        std = std.unsqueeze(0)
    
    return torch.clamp(tensor * std + mean, 0, 1)

print("✓ 反归一化函数定义完成")

### 3.2 展示训练样本

In [ ]:
def show_batch(dataloader, base_dataset, num_images=12, cols=4):
    """
    展示一个批次的图像
    
    Args:
        dataloader: 数据加载器
        base_dataset: 基础数据集（用于获取类别名称）
        num_images: 显示的图像数量
        cols: 每行显示的列数
    """
    # 获取一个批次
    images, labels = next(iter(dataloader))
    
    # 计算行数
    rows = (num_images + cols - 1) // cols
    
    # 创建图形
    fig, axes = plt.subplots(rows, cols, figsize=(cols*3, rows*3))
    axes = axes.flatten() if isinstance(axes, np.ndarray) else [axes]
    
    for idx in range(min(num_images, len(images))):
        # 反归一化
        img = denormalize_clip(images[idx].cpu())
        
        # 转换为numpy [C,H,W] -> [H,W,C]
        img_np = img.permute(1, 2, 0).numpy()
        
        # 获取类别信息
        label = labels[idx].item()
        wnid = base_dataset.get_wnid(label)
        class_name = base_dataset.get_class_name(label)
        
        # 显示图像
        axes[idx].imshow(img_np)
        axes[idx].set_title(f"{wnid}\n{class_name}", fontsize=9)
        axes[idx].axis('off')
    
    # 隐藏多余的子图
    for idx in range(num_images, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("训练集样本可视化:")
print("="*70)
show_batch(train_loader, base_dataset, num_images=12, cols=4)

### 3.3 展示验证集样本

In [ ]:
print("验证集样本可视化:")
print("="*70)
show_batch(val_loader, base_dataset, num_images=12, cols=4)

## 4. 样本详细信息

查看数据集中样本的详细信息（路径、WNID、类别名称、描述）。

In [ ]:
print("样本详细信息示例:")
print("="*70)

# 显示前5个样本的详细信息
for i in range(5):
    info = base_dataset.get_sample_info(i)
    print(f"\n样本 {i+1}:")
    print(f"  文件: {os.path.basename(info['path'])}")
    print(f"  WNID: {info['wnid']}")
    print(f"  标签索引: {info['label']}")
    print(f"  类别名称: {info['class_name']}")
    print(f"  描述: {info['description']}")

## 5. 直接使用Dataset类（高级用法）

如果需要更细粒度的控制，可以直接使用`ImageNetDataset`类。

In [ ]:
# 自定义transform
custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.48145466, 0.4578275, 0.40821073),
        std=(0.26862954, 0.26130258, 0.27577711)
    ),
])

# 创建自定义数据集
custom_dataset = ImageNetDataset(
    root=DATA_ROOT,
    split='train',
    transform=custom_transform,
    num_classes=5,  # 只使用前5个类别
)

print(f"自定义数据集大小: {len(custom_dataset)}")
print(f"类别数: {custom_dataset.num_classes}")

# 获取一个样本
img, label = custom_dataset[0]
print(f"\n样本形状: {img.shape}")
print(f"标签: {label}")
print(f"类别: {custom_dataset.get_class_name(label)}")

## 6. 指定特定类别

可以通过WNID列表指定想要使用的类别。

In [ ]:
# 指定感兴趣的类别（例如：动物类）
selected_wnids = [
    "n01443537",  # goldfish
    "n01530575",  # brambling
    "n01729322",  # hognose snake
    "n01773797",  # garden spider
    "n02480495",  # orangutan (如果有的话)
]

# 创建指定类别的数据加载器
print("创建指定类别的数据加载器...")
subset_train_loader, subset_val_loader = create_imagenet_dataloaders(
    root=DATA_ROOT,
    batch_size=BATCH_SIZE,
    num_workers=0,
    class_subset=selected_wnids,  # 指定类别
    image_size=IMAGE_SIZE,
    use_clip_norm=True,
    seed=SEED,
    val_split=0.15,
)

# 获取基础dataset
subset_base = subset_train_loader.dataset
if hasattr(subset_base, 'dataset'):
    subset_base = subset_base.dataset

print(f"\n✓ 子集类别数: {subset_base.num_classes}")
print(f"\n类别列表:")
for i in range(subset_base.num_classes):
    print(f"  [{i}] {subset_base.get_wnid(i)}: {subset_base.get_class_name(i)}")

## 7. 数据统计

分析数据集的类别分布。

In [ ]:
from collections import Counter

# 统计每个类别的样本数量
label_counts = Counter()
for _, label in base_dataset:
    label_counts[label] += 1

print("类别样本数量统计:")
print("="*70)
for label in sorted(label_counts.keys()):
    wnid = base_dataset.get_wnid(label)
    name = base_dataset.get_class_name(label)
    count = label_counts[label]
    print(f"  [{label:2d}] {wnid}: {name:40s} - {count:5d} 张")

# 可视化分布
labels = list(label_counts.keys())
counts = [label_counts[l] for l in labels]
names = [base_dataset.get_class_name(l).split(',')[0][:20] for l in labels]

plt.figure(figsize=(14, 6))
plt.bar(range(len(labels)), counts)
plt.xlabel('类别索引')
plt.ylabel('样本数量')
plt.title('ImageNet类别样本分布')
plt.xticks(range(len(labels)), labels, rotation=45)
plt.tight_layout()
plt.show()

## 总结

✅ **成功创建了基于 `nn.Module` 的ImageNet数据集类！**

### 主要特性：

1. **标准PyTorch接口**：继承自`torch.utils.data.Dataset`
2. **自动元数据解析**：从`meta.mat`自动加载WNID和类别名称
3. **灵活的类别选择**：支持按数量或指定WNID列表选择类别
4. **智能数据分割**：自动处理有/无独立val目录的情况
5. **CLIP集成**：内置CLIP预处理和归一化
6. **可重复性**：支持随机种子设置

### 使用方法：

```python
from imagenet_dataset import create_imagenet_dataloaders

train_loader, val_loader = create_imagenet_dataloaders(
    root='path/to/imagenet',
    batch_size=32,
    num_classes=20,
    use_clip_norm=True,
    seed=42
)
```